In [111]:
import numpy as np
import pandas as pd

In [112]:
df = pd.read_csv("./penguins.csv")
df.head()

,Species,CulmenLength,CulmenDepth,FlipperLength,OriginLocation,BodyMass
0,Adelie,39.1,18.7,181.0,Torgersen,3750.0
1,Adelie,39.5,17.4,186.0,Torgersen,3800.0
2,Adelie,40.3,18.0,195.0,Torgersen,3250.0
3,Adelie,NaN,NaN,NaN,Torgersen,NaN
4,Adelie,36.7,19.3,193.0,Torgersen,3450.0


In [113]:
df['Species'].unique()

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Species         150 non-null    object 
 1   CulmenLength    148 non-null    float64
 2   CulmenDepth     148 non-null    float64
 3   FlipperLength   148 non-null    float64
 4   OriginLocation  150 non-null    object 
 5   BodyMass        148 non-null    float64
dtypes: float64(4), object(2)
memory usage: 7.2+ KB


In [115]:
df.describe()

,CulmenLength,CulmenDepth,FlipperLength,BodyMass
count,148.000000,148.000000,148.000000,148.000000
mean,45.089865,17.377703,199.277027,4186.486486
std,5.385578,2.072129,14.160797,817.476925
min,34.100000,13.100000,172.000000,2700.000000
25%,40.500000,15.700000,187.750000,3550.000000
50%,46.100000,17.800000,196.000000,3950.000000
75%,49.350000,18.900000,212.250000,4681.250000
max,59.600000,21.500000,230.000000,6300.000000


In [116]:
print(df[df.isna()==False])

    Species  CulmenLength  CulmenDepth  FlipperLength OriginLocation  BodyMass
0    Adelie          39.1         18.7          181.0      Torgersen    3750.0
1    Adelie          39.5         17.4          186.0      Torgersen    3800.0
2    Adelie          40.3         18.0          195.0      Torgersen    3250.0
3    Adelie           NaN          NaN            NaN      Torgersen       NaN
4    Adelie          36.7         19.3          193.0      Torgersen    3450.0
..      ...           ...          ...            ...            ...       ...
145  Gentoo           NaN          NaN            NaN         Biscoe       NaN
146  Gentoo          46.8         14.3          215.0         Biscoe    4850.0
147  Gentoo          50.4         15.7          222.0         Biscoe    5750.0
148  Gentoo          45.2         14.8          212.0         Biscoe    5200.0
149  Gentoo          49.9         16.1          213.0         Biscoe    5400.0

[150 rows x 6 columns]


In [117]:
df.isna().sum()

Species           0
CulmenLength      2
CulmenDepth       2
FlipperLength     2
OriginLocation    0
BodyMass          2
dtype: int64

In [118]:
def fill_nulls(df):
    mask = df['Species'] == 'Gentoo'
    df.loc[mask] = df.loc[mask].fillna(df.loc[mask].mean(numeric_only=True))

    mask = df['Species'] == 'Adelie'
    df.loc[mask] = df.loc[mask].fillna(df.loc[mask].mean(numeric_only=True))
    
    mask = df['Species'] == 'Chinstrap'
    df.loc[mask] = df.loc[mask].fillna(df.loc[mask].mean(numeric_only=True))
    
    return df




In [119]:
df = fill_nulls(df)

In [120]:
df.head()

,Species,CulmenLength,CulmenDepth,FlipperLength,OriginLocation,BodyMass
0,Adelie,39.10000,18.700000,181.000000,Torgersen,3750.000000
1,Adelie,39.50000,17.400000,186.000000,Torgersen,3800.000000
2,Adelie,40.30000,18.000000,195.000000,Torgersen,3250.000000
3,Adelie,38.82449,18.767347,186.591837,Torgersen,3696.428571
4,Adelie,36.70000,19.300000,193.000000,Torgersen,3450.000000


In [121]:
df['OriginLocation'].value_counts()

OriginLocation
Dream        70
Biscoe       60
Torgersen    20
Name: count, dtype: int64

In [122]:

# keep mapping to decode later
species_mapping = dict(enumerate(df['Species'].astype('category').cat.categories))
# Label-encode Species (0...n-1)
df['Species'] = df['Species'].astype('category').cat.codes


In [123]:
print('Species mapping:', species_mapping)
print(df['Species'].head())

Species mapping: {0: 'Adelie', 1: 'Chinstrap', 2: 'Gentoo'}
0    0
1    0
2    0
3    0
4    0
Name: Species, dtype: int8


In [124]:
# One-hot encode OriginLocation
origin_dummies = pd.get_dummies(df['OriginLocation'], prefix='Origin', dummy_na=False)
df = pd.concat([df, origin_dummies], axis=1)


In [125]:
print(df.filter(like='Origin_').head())

   Origin_Biscoe  Origin_Dream  Origin_Torgersen
0          False         False              True
1          False         False              True
2          False         False              True
3          False         False              True
4          False         False              True


In [126]:
df.drop(columns=['OriginLocation'],inplace=True)

In [127]:
X = df.drop(columns=['Species'])
y = df['Species']
X.head()

,CulmenLength,CulmenDepth,FlipperLength,BodyMass,Origin_Biscoe,Origin_Dream,Origin_Torgersen
0,39.10000,18.700000,181.000000,3750.000000,False,False,True
1,39.50000,17.400000,186.000000,3800.000000,False,False,True
2,40.30000,18.000000,195.000000,3250.000000,False,False,True
3,38.82449,18.767347,186.591837,3696.428571,False,False,True
4,36.70000,19.300000,193.000000,3450.000000,False,False,True
